# Imports and setting up viz

NB : conda env1 on Mac, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *

In [2]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load files and edit datasets

LAM output analysis.

Simu irr et simu no-irr.

In [3]:
# lon_min=-10
# lon_max=4
# lat_min=34
# lat_max=46

In [4]:
# noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/noirr_2010_2022'
# irr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/irr_2010_2022'

# noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60_sim248'
# irr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60_sim258'

noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/noirr'
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/irr'

## LMDZ

In [ ]:
filename = '{}/*/ATM/MO/*.nc'.format(irr_dir)

lmdz_full = xr.open_mfdataset(filename)
lmdz_full = lmdz_full.rename({'time_counter':'time'})
lmdz_full.attrs['name'] = 'no_irr'
lmdz_full = lmdz_full.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
lmdz_full

In [6]:
LMDZ_TS=True

In [ ]:
#open histmth files
if LMDZ_TS:
    filename = '{}/*/ATM/TS_MO/*.nc'.format(noirr_dir)
    # filename = '{}/ATM/TS_MO*.nc'.format(noirr_dir)
else:
    filename = '{}/*/ATM/MO/*.nc'.format(noirr_dir)

lmdz_noirr0 = xr.open_mfdataset(filename)
lmdz_noirr = lmdz_noirr0.rename({'time_counter':'time'})
lmdz_noirr.attrs['name'] = 'no_irr'
lmdz_noirr.attrs['plot_color']='red'
lmdz_noirr = lmdz_noirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

lmdz_noirr['evap']=lmdz_noirr['evap'] * 3600 * 24
lmdz_noirr['evap'].attrs['units']='mm/d'
lmdz_noirr['evap'].attrs['long_name'] = 'Evapotranspiration'

lmdz_noirr['precip']=lmdz_noirr['precip'] * 3600 * 24
lmdz_noirr['precip'].attrs['long_name'] = 'Total precipitation'
lmdz_noirr['precip'].attrs['units']='mm/d'

lmdz_noirr['sens']=lmdz_noirr['sens']
lmdz_noirr['flat']=lmdz_noirr['flat']

lmdz_noirr

In [ ]:
if LMDZ_TS:
    filename = '{}/*/ATM/TS_MO/*.nc'.format(irr_dir)
else:
    filename = '{}/*/ATM/MO/*.nc'.format(irr_dir)

lmdz_irr0 = xr.open_mfdataset(filename)
lmdz_irr =lmdz_irr0.rename({'time_counter':'time'})
lmdz_irr.attrs['name'] = 'irr'
lmdz_irr.attrs['plot_color']='#0C5DA5'
lmdz_irr = lmdz_irr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

lmdz_irr['evap']=lmdz_irr['evap'] * 3600 * 24
lmdz_irr['evap'].attrs['units']='mm/d'
lmdz_irr['precip']=lmdz_irr['precip'] * 3600 * 24
lmdz_irr['precip'].attrs['units']='mm/d'
lmdz_irr['sens']=-lmdz_irr['sens']
lmdz_irr['flat']=-lmdz_irr['flat']

lmdz_irr

In [ ]:
if LMDZ_TS:
    # drop unused coordinates
    lmdz_noirr = lmdz_noirr.drop_vars('presnivs')
    lmdz_noirr = lmdz_noirr.drop_vars('time_centered')
    lmdz_irr = lmdz_irr.drop_vars('presnivs')
    lmdz_irr = lmdz_irr.drop_vars('time_centered')
    print('using TimeSeries outputs')

else:
    #add var t1 to lmdz datasets (first level of temp)
    lmdz_noirr['t1']    = lmdz_noirr['temp'][:,0,:,:]
    lmdz_noirr['t1'].attrs['long_name'] = 'Temperature at first level'
    lmdz_noirr['t1'].attrs['units']='K'
    
    lmdz_irr['t1']      = lmdz_irr['temp'][:,0,:,:]
    lmdz_irr['t1'].attrs['long_name'] = 'Temperature at first level'
    lmdz_irr['t1'].attrs['units']='K'
    print('using full outputs')

## ORCHIDEE

In [ ]:
filename = '{}/*/SRF/MO/*history.nc'.format(irr_dir)

orc_full = xr.open_mfdataset(filename)
orc_full = orc_full.rename({'time_counter':'time'})
orc_full.attrs['name'] = 'irr'
orc_full = orc_full.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

orc_full['irrig_frac'] = orc_full['irrigmap_dyn']/orc_full['Areas'] * 100
orc_full['irrig_frac'].attrs['units'] = '%'

orc_full

In [ ]:
# filename = '{}/*/SRF/MO/*sechiba_history.nc'.format(noirr_dir)
filename = '{}/*/SRF/TS_MO/*.nc'.format(noirr_dir)

orc_noirr0 = xr.open_mfdataset(filename)
orc_noirr = orc_noirr0.rename({'time_counter':'time'})
orc_noirr.attrs['name'] = 'no_irr'
orc_noirr.attrs['plot_color']='red'
orc_noirr = orc_noirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
orc_noirr

In [ ]:
# filename = '{}/*/SRF/MO/*history.nc'.format(irr_dir)
filename = '{}/*/SRF/TS_MO/*.nc'.format(irr_dir)

orc_irr0 = xr.open_mfdataset(filename)
orc_irr = orc_irr0.rename({'time_counter':'time'})
orc_irr.attrs['name'] = 'irr'
orc_irr.attrs['plot_color']='#0C5DA5'
orc_irr = orc_irr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

orc_irr

## Period selection

In [13]:
# Select months
months=[6,7,8]
# lmdz_irr =lmdz_irr.sel(time=lmdz_irr['time.month'].isin(months))
# lmdz_noirr = lmdz_noirr.sel(time=lmdz_noirr['time.month'].isin(months))
# lmdz_full = lmdz_full.sel(time=lmdz_full['time.month'].isin(months))

# orc_irr = orc_irr.sel(time=orc_irr['time.month'].isin(months))
# orc_noirr = orc_noirr.sel(time=orc_noirr['time.month'].isin(months))
# orc_full = orc_full.sel(time=orc_full['time.month'].isin(months))

## Masking

In [14]:
#create a masks of data points where irrig_frac >5%
irr_mask = (orc_full['irrigmap_dyn']/orc_full['Areas'])>0.05
con_mask_ORC=orc_full['Contfrac']>0.9
ip_mask_ORC=polygon_to_mask(orc_noirr, iberian_peninsula)

In [15]:
#iberian peninsula ds
ip_orc_noirr    = orc_noirr.where(con_mask_ORC).where(ip_mask_ORC['mask'])
ip_orc_irr      = orc_irr.where(con_mask_ORC).where(ip_mask_ORC['mask'])

In [16]:
ip_lmdz_noirr   = lmdz_noirr.where(con_mask_ORC).where(ip_mask_ORC['mask'])
ip_lmdz_irr     = lmdz_irr.where(con_mask_ORC).where(ip_mask_ORC['mask'])

In [17]:
#irrigated vs nonirr areas
irr_orc_noirr=orc_noirr.where(irr_mask)
irr_orc_irr=orc_irr.where(irr_mask)
irr_lmdz_noirr=lmdz_noirr.where(irr_mask)
irr_lmdz_irr=lmdz_irr.where(irr_mask)

irr_ip_orc_noirr=ip_orc_noirr.where(irr_mask)
irr_ip_orc_irr=ip_orc_irr.where(irr_mask)
irr_ip_lmdz_noirr=ip_lmdz_noirr.where(irr_mask)
irr_ip_lmdz_irr=ip_lmdz_irr.where(irr_mask)

#non irrigated areas
nonirr_orc_noirr=orc_noirr.where(~irr_mask)
nonirr_orc_irr=orc_irr.where(~irr_mask)
nonirr_lmdz_noirr=lmdz_noirr.where(~irr_mask)
nonirr_lmdz_irr=lmdz_irr.where(~irr_mask)

nonirr_ip_orc_noirr=ip_orc_noirr.where(~irr_mask)
nonirr_ip_orc_irr=ip_orc_irr.where(~irr_mask)
nonirr_ip_lmdz_noirr=ip_lmdz_noirr.where(~irr_mask)
nonirr_ip_lmdz_irr=ip_lmdz_irr.where(~irr_mask)

In [18]:
#for both full datasets
ip_orc_full=orc_full.where(ip_mask_ORC['mask']).where(con_mask_ORC)
ip_lmdz_full=lmdz_full.where(ip_mask_ORC['mask']).where(con_mask_ORC)
irr_ip_orc_full=ip_orc_full.where(irr_mask)

## Mean and difference datasets

In [19]:
orc_noirr_mean=orc_noirr.mean(dim='time')
orc_noirr_mean.attrs['name']='ORC no_irr'
orc_irr_mean=orc_irr.mean(dim='time')
orc_irr_mean.attrs['name']='ORC irr'

lmdz_noirr_mean=lmdz_noirr.mean(dim='time')
lmdz_noirr_mean.attrs['name']=' LMDZ no_irr'
lmdz_irr_mean=lmdz_irr.mean(dim='time')
lmdz_irr_mean.attrs['name']='LMDZ irr'

#same for IP
ip_orc_noirr_mean=ip_orc_noirr.mean(dim='time')
ip_orc_noirr_mean.attrs['name']='IP ORC no_irr'
ip_orc_irr_mean=ip_orc_irr.mean(dim='time')
ip_orc_irr_mean.attrs['name']='IP ORC irr'

ip_lmdz_noirr_mean=ip_lmdz_noirr.mean(dim='time')
ip_lmdz_noirr_mean.attrs['name']='IP LMDZ no_irr'
ip_lmdz_irr_mean=ip_lmdz_irr.mean(dim='time')
ip_lmdz_irr_mean.attrs['name']='IP LMDZ irr'


In [20]:
#Full
orc_full_mean=orc_full.mean(dim='time')
orc_full_mean.attrs['name']='ORC irr'
lmdz_full_mean=lmdz_full.mean(dim='time')
lmdz_full_mean.attrs['name']='LMDZ irr'

ip_orc_full_mean=orc_full.mean(dim='time')
ip_orc_full_mean.attrs['name']='IP ORC irr'
ip_lmdz_full_mean=lmdz_full.mean(dim='time')
ip_lmdz_full_mean.attrs['name']='IP LMDZ irr'

In [21]:
#irr - noirr
orc_diff = orc_irr - orc_noirr
orc_diff.attrs['name'] = 'irr - noirr'
lmdz_diff =lmdz_irr - lmdz_noirr
lmdz_diff.attrs['name'] = 'irr - noirr'

#mean
orc_diff_mean = orc_diff.mean(dim='time')
orc_diff_mean.attrs['name'] = 'irr - noirr mean'
lmdz_diff_mean = lmdz_diff.mean(dim='time')
lmdz_diff_mean.attrs['name'] = 'irr - noirr mean'

In [22]:
#over IP
ip_orc_diff = ip_orc_irr - ip_orc_noirr
ip_orc_diff.attrs['name'] = 'irr - noirr'
ip_orc_diff_mean = ip_orc_diff.mean(dim='time')
ip_orc_diff_mean.attrs['name'] = 'irr - noirr mean'

ip_lmdz_diff = ip_lmdz_irr - ip_lmdz_noirr
ip_lmdz_diff.attrs['name'] = 'irr - noirr'
ip_lmdz_diff_mean = ip_lmdz_diff.mean(dim='time')
ip_lmdz_diff_mean.attrs['name'] = 'irr - noirr mean'

# Maps

In [ ]:
var='irrigation'
ds=irr_ip_orc_full
vmin=-0.0
vmax= 1.5
# vmin,vmax = None, None
cmap=wet
map_ave(ds, var, vmin=vmin, vmax=vmax, cmap=cmap, hex=False)

In [24]:
plt.rcParams['hatch.color']='red'
plt.rcParams['hatch.color']='dimgrey'
plt.rcParams['hatch.linewidth'] = 1

In [25]:
#Diff map
var='evap'
ds1=ip_lmdz_irr
ds2=ip_lmdz_noirr
vmax= 3E-2
vmin=-3E-2
# vmin,vmax = None, None
cmap=emb
pvalue_limit=0.2

In [ ]:
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, 
             cmap=cmap,  sig=True, pvalue_limit=pvalue_limit, shade=True, hatch='xxx')

In [ ]:
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap,hex=False, sig=True, pvalue_limit=pvalue_limit)

In [ ]:
brr

In [ ]:
#pvalue plot
var='sens'
max_value=0.2
min_value=0
min_value, max_value= None, None
ds1=lmdz_irr
ds2=lmdz_noirr
p_values = xr.apply_ufunc(
    lambda x, y: ttest_ind(x, y, axis=0, nan_policy='omit').pvalue, 
    ds1[var], ds2[var],
    input_core_dims=[['time'], ['time']],
    output_core_dims=[[]],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
    dask_gufunc_kwargs={'allow_rechunk': True}
)
title='pvalues for {}, JJA'.format(var)
map_plotvar(p_values, vmin=min_value, vmax=max_value, cmap=wet,title=title)

In [ ]:
# Display 2 maps of var
var = 'sens'
ds1 = lmdz_irr
ds2 = lmdz_noirr
color_map=reds

map_two_ds(ds1, ds2, var, in_cmap=color_map)

In [ ]:
#Relative diff map
var='sfcWind'
maxvalue=None
ds1=lmdz_irr
ds2=lmdz_noirr

map_rel_diff_ave(ds1, ds2, var, in_vmax=maxvalue)

In [ ]:
#Display map of var for ds on restricted area
var='t2m'
ds = lmdz_noirr
# ds = std
latmin=36.0
latmax=44
lonmin=-10
lonmax=3.5

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')

map_plotvar(plotvar, in_cmap=emb_neutral, in_vmax=None, title=var)

# Time series and seasonnal cycle

In [ ]:
var='precip'

ds1=lmdz_noirr
ds2=lmdz_irr
ds_list=[ds1, ds2]

year_max=2022
time_series_ave(ds_list, var, year_max=year_max, title='{} ({})'.format(var, ds1[var].attrs['units']))
seasonal_cycle_ave(ds_list, var, year_max=year_max, title='Seasonal cycle of {} ({})'.format(var, ds1[var].attrs['units']))

# Scatter plots

In [ ]:
var1='irrigation'
ds1=ip_orc_full_mean

var2='evap'
ds2=ip_orc_diff_mean

title='Annual mean (1 data pt = 1 grid cell)'

scatter_vars(ds1, ds2, var1, var2, reg=True, plot_one=False, title=title)

In [ ]:
var1='convoccur'
ds1=ip_lmdz_diff_mean

var2='rain'
ds2=ip_orc_diff_mean

scatter_vars(ds1, ds2, var1, var2, reg=True, plot_one=False)

## Spatial aggregation

In [ ]:
#define new datasets with spatial aggregation over domain
ip_orc_full_aggr = ip_orc_full.mean(dim='lon').mean(dim='lat')
ip_lmdz_full_aggr = ip_lmdz_full.mean(dim='lon').mean(dim='lat')
ip_orc_irr_aggr = ip_orc_irr.mean(dim='lon').mean(dim='lat')
ip_lmdz_irr_aggr = ip_lmdz_irr.mean(dim='lon').mean(dim='lat')
ip_orc_noirr_aggr = ip_orc_noirr.mean(dim='lon').mean(dim='lat')
ip_lmdz_noirr_aggr = ip_lmdz_noirr.mean(dim='lon').mean(dim='lat')
ip_orc_diff_aggr = ip_orc_diff.mean(dim='lon').mean(dim='lat')
ip_lmdz_diff_aggr = ip_lmdz_diff.mean(dim='lon').mean(dim='lat')

irr_ip_orc_full_aggr = irr_ip_orc_full.mean(dim='lon').mean(dim='lat')
# irr_ip_lmdz_full_aggr = irr_ip_lmdz_full.mean(dim='lon').mean(dim='lat')
irr_ip_orc_irr_aggr = irr_ip_orc_irr.mean(dim='lon').mean(dim='lat')
irr_ip_lmdz_irr_aggr = irr_ip_lmdz_irr.mean(dim='lon').mean(dim='lat')
irr_ip_orc_noirr_aggr = irr_ip_orc_noirr.mean(dim='lon').mean(dim='lat')
irr_ip_lmdz_noirr_aggr = irr_ip_lmdz_noirr.mean(dim='lon').mean(dim='lat')
irr_ip_orc_diff_aggr = irr_ip_orc_diff.mean(dim='lon').mean(dim='lat')
irr_ip_lmdz_diff_aggr = irr_ip_lmdz_diff.mean(dim='lon').mean(dim='lat')

#name new datasets
ip_orc_full_aggr.attrs['name']='ORC irr IP mean'
ip_lmdz_full_aggr.attrs['name']='LMDZ irr IP mean'
ip_orc_irr_aggr.attrs['name']='ORC irr IP mean'
ip_lmdz_irr_aggr.attrs['name']='LMDZ irr IP mean'
ip_orc_noirr_aggr.attrs['name']='ORC no_irr IP mean'
ip_lmdz_noirr_aggr.attrs['name']='LMDZ no_irr IP mean'
ip_orc_diff_aggr.attrs['name']='ORC diff IP mean'
ip_lmdz_diff_aggr.attrs['name']='LMDZ diff IP mean'



In [ ]:
#add irr and nonirr distinction for diff datasets
irr_ip_orc_full_aggr = ip_orc_full_aggr.where(irr_mask)

irr_ip_orc_diff_aggr = irr_ip_orc_diff.mean(dim='lon').mean(dim='lat') #todo!
nonirr_ip_orc_diff_aggr = ip_orc_diff_aggr.where(~irr_mask)
irr_ip_lmdz_diff_aggr = ip_lmdz_diff_aggr.where(irr_mask)
nonirr_ip_lmdz_diff_aggr = ip_lmdz_diff_aggr.where(~irr_mask)

irr_ip_orc_full_aggr.attrs['name']='ORC irr IP mean'
irr_ip_orc_diff_aggr.attrs['name']='ORC diff IP irrZone mean'
nonirr_ip_orc_diff_aggr.attrs['name']='ORC diff IP nonirrZone mean'
irr_ip_lmdz_diff_aggr.attrs['name']='LMDZ diff IP irrZone mean'
nonirr_ip_lmdz_diff_aggr.attrs['name']='LMDZ diff IP nonirrZone mean'

In [ ]:
var1='irrigation'
ds1=irr_ip_orc_full_aggr

var2='rain'
ds2=irr_ip_orc_diff_aggr

title='Irrigated areas domain mean (1 data pt = 1 sim month)'

scatter_vars(ds1, ds2, var1, var2, reg=True, plot_one=True, title=title)